In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import gc
import time
gc.enable()
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re
import geopy
from tqdm import tqdm_notebook as tqdm
import json
import os

warnings.filterwarnings('ignore')

In [2]:
%%time
# Load from preprocessed data
df_2018 = pd.read_feather('../data/Final_Divvy_data_2018.feather')
df_2017 = pd.read_feather('../data/Final_Divvy_data_2017.feather')
df_2016 = pd.read_feather('../data/Final_Divvy_data_2016.feather')
df_2015 = pd.read_feather('../data/Final_Divvy_data_2015.feather')
df_2014 = pd.read_feather('../data/Final_Divvy_data_2014.feather')
df_2013 = pd.read_feather('../data/Final_Divvy_data_2013.feather')

CPU times: user 333 ms, sys: 1.04 s, total: 1.37 s
Wall time: 1.69 s


In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import widgets
import plotly.graph_objs as go

init_notebook_mode(connected=True)

## Load station data of 2013-2018
[Find your plotly colorscale](https://plot.ly/python/colorscales/)

In [4]:
# function to group total trip data over station
def group_over_dow_m(df):
    tmp_df = df[['month', 'dayofweek', 'year', 'total_in']]\
                .groupby(['month', 'dayofweek'])\
                .agg({'year': np.mean, 'total_in': np.sum}).reset_index()
    return np.array(tmp_df.total_in).reshape(12, 7).T

In [5]:
# list of months and dow
mx = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
dwy = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [6]:
def get_plot_info(df):
    tripcount = group_over_dow_m(df)
    hover_text = []
    for i in range(7):
        hy = dwy[i]
        tmp = []
        for j in range(12):
            hx = mx[j]
            tmp.append('Month: '+hx+'<br>Day of Week: '+hy+'<br>Total trips: '+str(int(tripcount[i][j])))
        hover_text.append(tmp)
    return tripcount, hover_text

In [7]:
# Prepare data
zmin = 0
zmax = 100000
cs = 'Hot'
tp2018, ht2018 = get_plot_info(df_2018)
tp2017, ht2017 = get_plot_info(df_2017)
tp2016, ht2016 = get_plot_info(df_2016)
tp2015, ht2015 = get_plot_info(df_2015)
tp2014, ht2014 = get_plot_info(df_2014)
trace2018 = go.Heatmap(
    z=tp2018,
    x=mx,
    y=dwy,
    text=ht2018,
    hoverinfo='text',
    colorscale=cs,
    reversescale=True,
    zmin = zmin,
    zmax = zmax,
    zauto=False,
    visible=False
)
trace2017 = go.Heatmap(
    z=tp2017,
    x=mx,
    y=dwy,
    text=ht2017,
    hoverinfo='text',
    colorscale=cs,
    reversescale=True,
    zmin = zmin,
    zmax = zmax,
    zauto=False,
    visible=False
)
trace2016 = go.Heatmap(
    z=tp2016,
    x=mx,
    y=dwy,
    text=ht2016,
    hoverinfo='text',
    colorscale=cs,
    reversescale=True,
    zmin = zmin,
    zmax = zmax,
    zauto=False,
    visible=False
)
trace2015 = go.Heatmap(
    z=tp2015,
    x=mx,
    y=dwy,
    text=ht2015,
    hoverinfo='text',
    colorscale=cs,
    reversescale=True,
    zmin = zmin,
    zmax = zmax,
    zauto=False,
    visible=False
)
trace2014 = go.Heatmap(
    z=tp2014,
    x=mx,
    y=dwy,
    text=ht2016,
    hoverinfo='text',
    colorscale=cs,
    reversescale=True,
    zmin = zmin,
    zmax = zmax,
    zauto=False 
)
data=[trace2014, trace2015, trace2016, trace2017, trace2018]

In [8]:
# Prepare update menu
updatemenus = list([
    dict(type="buttons",
         active=-1,
         buttons=list([
            dict(label = '2014',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False]}]),
            dict(label = '2015',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False]}]),
            dict(label = '2016',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False]}]),
            dict(label = '2017',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False]}]),
            dict(label = '2018',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True]}]),
        ]),
        direction = 'left',
        showactive = True,
        pad = {'r': 20, 't': 20},
        x = 0.62,
        xanchor = 'center',
        y = 1.1,
        yanchor = 'bottom',
        bgcolor = '#AAAAAA',
        bordercolor = '#FFFFFF',
        font = dict(size=18, color='#101010',family='Old Standard TT, serif')
    ),

])


In [9]:
# layout
layout = go.Layout(
    title=go.layout.Title(
        text='Total # of trips',
        xref='paper',
        x=0
    ),
    xaxis=dict(
        title='Month',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='grey'
        ),
        showticklabels=True,
        tickangle=-45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Day of Week',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='grey'
        ),
        showticklabels=True,
        tickangle=-45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    updatemenus=updatemenus
)

In [10]:
fig = go.Figure(data=data, layout=layout)
plot(fig, filename='trip_count_heatmap.html')

'trip_count_heatmap.html'